In [11]:
import json

import torch
from torch import nn
from torch.utils.data import Dataset

import numpy as np
import pandas as pd
import matplotlib.image as mpimg

# import loader

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
batch_size = 32
TRAIN_LABELS = 'data/train.json'
IMG_DIR = 'data/images'

In [13]:
### copy of loader.py

TRAIN_LABELS = 'data/train.json'
TEST_LABELS = 'data/test.json'
IMG_DIR = 'data/images'


def load_image(id, img_dir=IMG_DIR):
    return mpimg.imread(f'{img_dir}/punk{"%04d" % id}.png')


def write_image(img_data, filepath):
    return mpimg.imsave(filepath, img_data)


def make_punks_df(labels_file):
    with open(labels_file) as f:
        punks = json.loads(f.read())
    df = pd.DataFrame.from_dict(punks, orient='index')
    df['img'] = df.apply(lambda row: [load_image(int(row.name))], axis=1)
    return df


def any_to_one(i):
    x = True
    if i['any']:
        x = False
    return int(x)


class PunksDataset(Dataset):
    def __init__(self, filter, img_dir=IMG_DIR, train=False):
        self.filter = filter
        self.img_dir = img_dir

        self.labels_path = TRAIN_LABELS if train else TEST_LABELS
        punks_df = make_punks_df(self.labels_path)
        self.X = np.array([row[0] for row in punks_df['img'].to_numpy()])
        self.Y = punks_df[filter].to_numpy()
        # punks_df['any'] = punks_df[filter].apply(np.any, axis=1)
        # punks_df['none'] = punks_df.apply(lambda x: any_to_one(x), axis=1)
        # self.Y = punks_df[filter + ['none']].to_numpy()

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]
        # return gimme_one(self.punks_df, self.filter, idx)


In [14]:
punks_df = make_punks_df(TRAIN_LABELS)

filter = ['wildHair', 'cigarette']
# filter = ['male', 'female', 'cigarette']

labels = np.array(punks_df[filter])
data = np.array([load_image(row) for row in range(len(labels))])

In [15]:
train_dataset = PunksDataset(filter, train=True)
# train_dataset = PunksDataset(filter, train=False)


punk, labels = train_dataset[42]
print(punk.shape)
print("filter:", labels)
# loader.write_image(punk2, "42.jpg")

(24, 24, 4)
filter: [1 1]


In [16]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=False
)

punk_batch, labels_batch = next(iter(train_loader))
print(type(punk_batch))
print(type(punk_batch[0]))
print(type(punk_batch[0][0]))
print(type(punk_batch[0][0][0]))
# print(type(punk_batch[0][0][0][0]))
print(punk_batch.shape)
print(punk_batch[0].shape)
print("filter:", labels_batch[0].numpy())

write_image(punk_batch[0].numpy(), "0.jpg")

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.Size([32, 24, 24, 4])
torch.Size([24, 24, 4])
filter: [0 0]


In [23]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(24*24*4, 200),
    nn.ReLU(),
    nn.Linear(200, 150),
    nn.ReLU(),
    nn.Linear(150, len(filter)),
    nn.ReLU(),
    # nn.Softmax(dim=1)
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
# criterion = nn.NLLLoss()

model.to(device)


Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=2304, out_features=200, bias=True)
  (2): ReLU()
  (3): Linear(in_features=200, out_features=150, bias=True)
  (4): ReLU()
  (5): Linear(in_features=150, out_features=2, bias=True)
  (6): ReLU()
)

In [24]:
num_epoch = 10
running_loss = 0


for epoch in range(num_epoch):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.type(torch.float64))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print('Epoch[{}/{}], loss: {:.6f}'
        .format(epoch + 1, num_epoch, loss.data.item()))

Epoch[1/10], loss: 0.011686
Epoch[2/10], loss: 0.001327
Epoch[3/10], loss: 0.000614
Epoch[4/10], loss: 0.001264
Epoch[5/10], loss: 0.000383
Epoch[6/10], loss: 0.000279
Epoch[7/10], loss: 0.000521
Epoch[8/10], loss: 0.000249
Epoch[9/10], loss: 0.000558
Epoch[10/10], loss: 0.000250


In [19]:
# # test_dataset = loader.PunksDataset(filter, train=True)
# test_dataset = loader.PunksDataset(filter, train=True)

# test_loader = torch.utils.data.DataLoader(
#     dataset=test_dataset, batch_size=batch_size, shuffle=False
# )

# img, label = next(iter(test_loader))
# loader.write_image(img[0].numpy(), 'test.jpg')

In [20]:
# outputs = model(img)
# print(outputs)
# vals = outputs.data.cpu().numpy().argmax()
# vals